## Настраиваем окружение

In [1]:
import folium
import pandas as pd
import numpy as np
import webbrowser

In [33]:
peregon = pd.read_excel(r"D:\MyFolder\2.Work\DataWagon\PEREGON_HACKATON1.xlsx")


In [3]:
station_coord = pd.read_excel(r"D:\MyFolder\2.Work\DataWagon\STATION_COORDS_HACKATON.xlsx")
disl = pd.read_excel("D:\MyFolder\\2.Work\DataWagon\disl_hackaton.xlsx")

In [23]:
station_coord_cleaned = station_coord.dropna()


In [34]:
peregon

,START_CODE,END_CODE,LEN
0,2,805,32
1,6,2,27
2,8,6,27
3,12,8,29
4,15,12,19
...,...,...,...
8969,24329,7173,9
8970,25137,5956,10
8971,25137,5985,1
8972,25612,1688,0


In [35]:
zero_len_rows = peregon[peregon['LEN'] == 0]

In [36]:
zero_len_rows

,START_CODE,END_CODE,LEN
8,30,29,0
19,68,10288,0
45,117,52,0
47,118,8174,0
107,278,276,0
...,...,...,...
8950,23026,7163,0
8954,23029,21546,0
8955,23030,1410,0
8966,23181,7031,0


In [37]:
# Удаление этих строк из оригинального датафрейма
peregon_clean = peregon.drop(zero_len_rows.index.to_list())

# Сброс индексов
peregon_clean = peregon.reset_index(drop=True)

In [38]:
peregon_clean

,START_CODE,END_CODE,LEN
0,2,805,32
1,6,2,27
2,8,6,27
3,12,8,29
4,15,12,19
...,...,...,...
8969,24329,7173,9
8970,25137,5956,10
8971,25137,5985,1
8972,25612,1688,0


## Проверка идеи

In [39]:
# Найти строки с NaN значениями
nan_rows = disl[disl.isnull().any(axis=1)]

# Вывести найденные строки
print(nan_rows)


        WAGNUM            OPERDATE  ST_ID_DISL  ST_ID_DEST TRAIN_INDEX
282814    6167 2023-06-12 23:50:00        9465        1227         NaN
282820    6167 2023-06-09 14:03:00       16947        1227         NaN
283042    6167 2023-06-11 18:52:00        9435        1227         NaN
283143    6167 2023-06-10 19:52:00        9448        1227         NaN
283650    6273 2023-06-12 23:50:00        9465        1227         NaN
283653    6273 2023-06-09 14:03:00       16947        1227         NaN
283980    6273 2023-06-11 18:52:00        9435        1227         NaN
284097    6273 2023-06-10 19:52:00        9448        1227         NaN
287940    7173 2023-06-12 23:50:00        9465        1227         NaN
287945    7173 2023-06-09 14:03:00       16947        1227         NaN
288283    7173 2023-06-11 18:52:00        9435        1227         NaN
288440    7173 2023-06-10 19:52:00        9448        1227         NaN
332566    9765 2023-07-22 14:00:00       20695        1561         NaN
332582

## Отрисовка карты

In [61]:
# Создаем карту
m = folium.Map(tiles="Cartodb dark_matter",location=[station_coord_cleaned['LATITUDE'].mean(), station_coord_cleaned['LONGITUDE'].mean()], zoom_start=5)
# Добавляем статичные маркеры (CircleMarker) для каждого объекта
for index, row in station_coord_cleaned.iterrows():
    folium.CircleMarker(location=[row['LATITUDE'], row['LONGITUDE']],
                        radius=2,  # радиус круга (можно настроить)
                        color='white',  # цвет круга
                        fill=True,
                        fill_color='white',  # цвет заливки
                        fill_opacity=0.7,  # прозрачность заливки
                        popup=row['ST_ID']).add_to(m)
# Добавляем линии между соответствующими вершинами

for index, row in peregon.iterrows():
    start_station = station_coord_cleaned.loc[station_coord_cleaned['ST_ID'] == row['START_CODE']]
    end_station = station_coord_cleaned.loc[station_coord_cleaned['ST_ID'] == row['END_CODE']]
    
    if not start_station.empty and not end_station.empty:
        start_coords = [start_station['LATITUDE'].values[0], start_station['LONGITUDE'].values[0]]
        end_coords = [end_station['LATITUDE'].values[0], end_station['LONGITUDE'].values[0]]
        
        folium.PolyLine(locations=[start_coords, end_coords], color='white', weight=2, opacity=0.7).add_to(m)

# Сохраняем карту в HTML-файл
m.save('base_map.html')

## Эмулируем поступление данных

In [70]:
# Сортировка данных по столбцу OPERDATE
sorted_df = disl.sort_values(by='OPERDATE')
# Группировка данных по столбцу OPERDATE
grouped_df = sorted_df.groupby('OPERDATE')


In [85]:
first_data_input = sorted_df[0:2500].reset_index()

In [86]:
# Преобразуем столбцы с датами в формат datetime
first_data_input['OPERDATE'] = pd.to_datetime(first_data_input['OPERDATE'])

# 1. Взять самую новую информацию по каждому вагону
latest_data = first_data_input.sort_values(by='OPERDATE', ascending=False).groupby('WAGNUM').head(1)


In [88]:
latest_data

,index,WAGNUM,OPERDATE,ST_ID_DISL,ST_ID_DEST,TRAIN_INDEX
2499,213325,9239,2023-06-01 08:21:00,739,1221,686-880-1176
2498,257399,6519,2023-06-01 08:21:00,739,1221,686-880-1176
2497,271254,3059,2023-06-01 08:21:00,1753,1221,1771-945-1221
2496,271255,4436,2023-06-01 08:21:00,1753,1221,1771-945-1221
2495,119817,8255,2023-06-01 08:21:00,1753,1221,1771-945-1221
...,...,...,...,...,...,...
231,662716,2345,2023-06-01 01:02:00,1771,1771,-244-
230,556459,392,2023-06-01 01:02:00,1771,1771,-244-
163,396109,5515,2023-06-01 00:46:00,1771,1771,1770-686-1771
108,333828,7161,2023-06-01 00:30:00,1602,1602,20705-722-1602


In [87]:
latest_data.groupby('TRAIN_INDEX').head(1)

,index,WAGNUM,OPERDATE,ST_ID_DISL,ST_ID_DEST,TRAIN_INDEX
2499,213325,9239,2023-06-01 08:21:00,739,1221,686-880-1176
2497,271254,3059,2023-06-01 08:21:00,1753,1221,1771-945-1221
2489,88484,6394,2023-06-01 08:20:00,20713,633,20713-688-585
2488,216393,5289,2023-06-01 08:18:00,1765,1221,1771-854-1736
2487,537892,5518,2023-06-01 08:18:00,4796,1771,4801-290-1771
...,...,...,...,...,...,...
285,298601,8090,2023-06-01 01:14:00,1663,1228,1268-675-1663
232,447534,9732,2023-06-01 01:02:00,1771,1771,-244-
163,396109,5515,2023-06-01 00:46:00,1771,1771,1770-686-1771
108,333828,7161,2023-06-01 00:30:00,1602,1602,20705-722-1602


## Добавляем на карту поезда

In [66]:
for index, row in latest_data.iterrows():
    # Добавляем маркер текущего расположения вагона
    current_location = station_coord.loc[station_coord['ST_ID'] == row['ST_ID_DISL']]
    if not current_location.empty:
        popup_html=fr"""<div >
                        Номер поезда: {row['TRAIN_INDEX']}<br> 
                        Дата обновления: {row["OPERDATE"]}<br>
                        ID станции прибытия: {row['ST_ID_DEST']}<br>
                        ID текущей станции: {row['ST_ID_DISL']}<br>
                        Номер вагона: {row["WAGNUM"]}<br>
                        <button onclick="addDestinationMarker()">Добавить маркер пункта назначения</button>
                        </div>"""
        folium.Marker(location=[current_location['LATITUDE'].values[0],
                                current_location['LONGITUDE'].values[0]],
                    popup=folium.Popup(popup_html, min_width=500),
                    icon=folium.Icon(color='orange', icon='leaf', prefix='fa')).add_to(m)


# Сохраняем карту в HTML-файл
m.save('train_map.html')
webbrowser.open_new_tab('train_map.html')# открываем полученную карту в браузере

True

## Пока не нужный код

In [67]:
import networkx as nx

# Создаем граф на основе данных из вашего датафрейма
G = nx.Graph()

# Здесь вы добавляете ребра в граф
for index, row in peregon.iterrows():
    G.add_edge(row['START_CODE'], row['END_CODE'], weight=row['LEN'])

def find_shortest_path(graph, start, end):
    try:
        # Используем алгоритм Дейкстры для поиска кратчайшего пути
        shortest_path = nx.shortest_path(graph, source=start, target=end, weight='weight')
        # Получаем длину кратчайшего пути
        shortest_path_length = nx.shortest_path_length(graph, source=start, target=end, weight='weight')
        
        # Находим два альтернативных пути
        alternative_paths = list(nx.all_shortest_paths(graph, source=start, target=end, weight='weight'))[1:3]
        
        return shortest_path, shortest_path_length, alternative_paths
    except nx.NetworkXNoPath:
        return None, None, None

# Пример использования
start_point = 4063  # Задайте начальную точку
end_point = 4534    # Задайте конечную точку

shortest_path, shortest_path_length, alternative_paths = find_shortest_path(G, start_point, end_point)

print("Кратчайший путь:", shortest_path)
print("Длина кратчайшего пути:", shortest_path_length)
print("Альтернативные пути:", alternative_paths)


Кратчайший путь: [4063, 4023, 4013, 4011, 4008, 4005, 4004, 4003, 4001, 3987, 3984, 3983, 3981, 3980, 3969, 3835, 3831, 7493, 3860, 3858, 3857, 4542, 4534]
Длина кратчайшего пути: 291
Альтернативные пути: []


In [68]:
result_df = pd.merge(station_coord_cleaned[["ST_ID", "LATITUDE", "LONGITUDE"]],
                    pd.DataFrame({"ST_ID": shortest_path}),
                    on="ST_ID")

# Преобразуйте DataFrame в список кортежей
coordinates_list = list(result_df[['LATITUDE', 'LONGITUDE']].itertuples(index=False, name=None))

print(coordinates_list)

[(55.1035, 61.3666), (55.0447, 61.2483), (55.184, 61.6217), (55.1633, 61.5405), (55.1516, 61.4655), (55.0293, 61.1107), (54.9832, 60.7566), (54.9965, 60.4447), (54.9846, 60.3618), (54.9859, 60.2958), (55.0334, 60.1111), (55.0577, 60.054), (55.0722, 59.9256), (55.084, 59.8258), (55.1176, 59.7675), (55.1982, 59.718), (55.1927, 59.6028), (55.2069, 59.5318), (55.1589, 59.1445), (55.1338, 58.8865), (55.2051, 62.2406), (55.1991, 61.7311), (55.1412, 61.4167)]


In [69]:
folium.PolyLine(locations=coordinates_list, color='orange', weight = 3, opacity=0.5).add_to(m)
folium.Marker(location=coordinates_list[0],popup=folium.Popup(popup_html, min_width=500),
                    icon=folium.Icon(color='green', icon='leaf', prefix='fa')).add_to(m)
folium.Marker(location=coordinates_list[-1],popup=folium.Popup(popup_html, min_width=500),
                    icon=folium.Icon(color='purple', icon='leaf', prefix='fa')).add_to(m)
m.save('map_with_line.html')
webbrowser.open_new_tab('map_with_line.html')


True